# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# websites=[f'https://books.toscrape.com/catalogue/page-{pag}.html' for pag in range(1,6)]
# websites

In [2]:
def scrape_books(min_rating, max_price, number_pages_to_scrape=51):
    websites=[f'https://books.toscrape.com/catalogue/page-{pag}.html' for pag in range(1, number_pages_to_scrape)]
    UPC_l=[]
    title_l=[]
    price_l=[]
    rating_l=[]
    genre_l=[]
    availability_l=[]
    description_l=[]
    for website in websites:
        pages_menu = requests.get(website)
        #pages_menu.content
        pages = BeautifulSoup(pages_menu.content, 'html.parser')
        pages=pages.find('ol', class_='row')
        pages=pages.find_all('a')
        books_links_list=set([page.get('href') for page in pages])
        books_links=['https://books.toscrape.com/catalogue/'+ link for link in books_links_list]
        
        for url in books_links:
            response = requests.get(url)
            response.content
            soup = BeautifulSoup(response.content, 'html.parser') 
    
            price=soup.find(string='Price (incl. tax)').find_next('td').text.strip()
            price=price.replace('£', '')
            price=float(price)
            # print(f'price {price}')
            
            rating_search=soup.find('p', class_='star-rating')
            rating_corelation={'One': 1,
                               'Two': 2,
                               'Three': 3,
                               'Four': 4,
                               'Five': 5}
            rating_2=rating_search.get('class')[1].strip()
            rating=rating_corelation.get(rating_2)
            rating=int(rating)
            # print(f'rating {rating}')
            
            if price<=max_price and rating>=min_rating:
                price_l.append(price)
                rating_l.append(rating)
                
                UPC=soup.find(class_="table table-striped").find_next(string='UPC').find_next('td').text.strip()
                UPC_l.append(UPC)
        
                title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
                title_l.append(title)
        
                genre=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
                genre_l.append(genre)
            
                availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
                availability=int(availability.split()[2].replace('(', ''))
                availability_l.append(availability)
        
                description=soup.find('div', class_='sub-header').find_next('p').text
                description_l.append(description)
                
    df=pd.DataFrame({'upc': UPC_l,
                     'title': title_l,
                     'price': price_l,
                     'rating': rating_l,
                     'genre': genre_l,
                     'availability': availability_l,
                     'description': description_l })
    return df

In [3]:
scrape_books(4, 20, 6)

,upc,title,price,rating,genre,availability,description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,19,Aaron Ledbetter’s future had been planned out ...
1,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,18,A page-turning novel that is also an explorati...
2,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,18,"In The Four Agreements, don Miguel Ruiz reveal..."
3,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,16,"Every summer, Rose goes with her mom and dad t..."
4,709822d0b5bcb7f4,Thirst,17.27,5,Fiction,16,"On a searing summer Friday, Eddie Chapman has ..."
5,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,16,"More than thirty-five years ago, when the weat..."
6,72f9d5be3472d34e,Mama Tried: Traditional Italian Cooking for th...,14.02,4,Food and Drink,16,Cecilia Granata grew up cooking with her famil...
7,0e691eda369f4e09,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Fantasy,16,Just as Annie and Liam are busy making plans t...
8,889139b8e9c4cb36,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Sequential Art,16,NEW HORROR SERIES FROM THE WALKING DEAD CREATO...
9,0fa6dceead7ce47a,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,16,THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...


In [4]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# response = requests.get('https://books.toscrape.com/catalogue/the-requiem-red_995/index.html')
# response.content

In [5]:
# soup = BeautifulSoup(response.content, 'html.parser')

UPC: The Universal Product Code (UPC) of the book.
Title: The title of the book.
Price (£): The price of the book in pounds.
Rating: The rating of the book (1-5 stars).
Genre: The genre of the book.
Availability: Whether the book is in stock or not.
Description: A brief description or product description of the book (if available).

def get_attributes(url):
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    response.content
    soup = BeautifulSoup(response.content, 'html.parser')

def UPC(soup):
    UPC=soup.find(class_="table table-striped").find_next(text='UPC').find_next('td').text.strip()
    return UPC
def title(soup):
    title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
    return title
def price(soup):
    price=soup.find(text='Price (incl. tax)').find_next('td').text.strip()
    price=price.replace('£', '')
    price=float(price)
    return price
def rating(soup):
    rating_search=soup.find('p', class_='star-rating')
    rating_corelation={'One': 1,
                   'Two': 2,
                   'Three': 3,
                   'Four': 4,
                   'Five': 5}
    rating_2=rating_search.get('class')[1].strip()
    rating=rating_corelation.get(rating_2)
    int(rating)
def genre(soup):
    genre=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
    return genre
def title(soup):
    availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
    return int(availability.split()[2].replace('(', ''))
def description(soup):
    description=soup.find('div', class_='sub-header').find_next('p').text
    return description

In [6]:
# UPC=soup.find(class_="table table-striped").find_next(text='UPC').find_next('td').text.strip()
# UPC

UPC_header = soup.find('th', string='UPC')

if UPC_header:
    UPC_data_cell = UPC_header.find_next_sibling('td')
    UPC = UPC_data_cell.text.strip()
    # Now UPC is safely extracted
else:
    # Handle the case where the UPC header is not found
    UPC = None
UPC

In [7]:
# title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
# title

In [8]:
# price=soup.find(text='Price (incl. tax)').find_next('td').text.strip()
# price=price.replace('£', '')
# price=float(price)
# price

In [9]:
# rating_search=soup.find('p', class_='star-rating')
# rating_corelation={'One': 1,
#                    'Two': 2,
#                    'Three': 3,
#                    'Four': 4,
#                    'Five': 5}
# rating_2=rating_search.get('class')[1].strip()
# rating=rating_corelation.get(rating_2)
# int(rating)

In [10]:
# genre_search=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
# genre_search

In [11]:
# genre_search=soup.find('a', string='Books').find_next('li').text.strip()
# genre_search

In [12]:
# availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
# int(availability.split()[2].replace('(', ''))

In [13]:
# description=soup.find('div', class_='sub-header').find_next('p').text
# description

websites=[f'https://books.toscrape.com/catalogue/page-{pag}.html' for pag in range(1,51)]
websites

pages_menu = requests.get('https://books.toscrape.com')
pages_menu.content
pages = BeautifulSoup(pages_menu.content, 'html.parser')
pages=pages.find('ol', class_='row')
pages=pages.find_all('a')
books_links_list=[page.get('href') for page in pages]
books_links=['https://books.toscrape.com/'+ link for link in books_links_list]
books_links

In [14]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# websites=[f'https://books.toscrape.com/catalogue/page-{pag}.html' for pag in range(1,51)]

def get_attributes(websites):
    UPC_l=[]
    title_l=[]
    price_l=[]
    rating_l=[]
    genre_l=[]
    availability_l=[]
    description_l=[]
    for website in websites:
        pages_menu = requests.get(website)
        pages_menu.content
        pages = BeautifulSoup(pages_menu.content, 'html.parser')
        pages=pages.find('ol', class_='row')
        pages=pages.find_all('a')
        books_links_list=[page.get('href') for page in pages]
        books_links=['https://books.toscrape.com/'+ link for link in books_links_list]
        
        for url in books_links:
            response = requests.get(url)
            response.content
            soup = BeautifulSoup(response.content, 'html.parser') 
            
            UPC=soup.find(class_="table table-striped").find_next(text='UPC').find_next('td').text.strip()
            UPC_l.append(UPC)

            title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
            title_l.append(title)

            price=soup.find(text='Price (incl. tax)').find_next('td').text.strip()
            price=price.replace('£', '')
            price=float(price)
            price_l.append(price)

            rating_search=soup.find('p', class_='star-rating')
            rating_corelation={'One': 1,
                           'Two': 2,
                           'Three': 3,
                           'Four': 4,
                           'Five': 5}
            rating_2=rating_search.get('class')[1].strip()
            rating=rating_corelation.get(rating_2)
            int(rating)
            rating.append(rating)

            genre=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
            genre_l.append(genre)

            availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
            availability=int(availability.split()[2].replace('(', ''))
            availability_l.append(availability)

            description=soup.find('div', class_='sub-header').find_next('p').text
            description_l.append(description)
    df=pd.DataFrame({'upc': UPC_l,
                     'title': title_l,
                     'price': price_l,
                     'rating': rating_l,
                     'genre': genre_l,
                     'availability': availability_l,
                     'description': description_l })
    return df

In [15]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# websites=[f'https://books.toscrape.com/catalogue/page-{pag}.html' for pag in range(1,6)]
# websites

In [16]:
# UPC_l=[]
# title_l=[]
# price_l=[]
# rating_l=[]
# genre_l=[]
# availability_l=[]
# description_l=[]
# for website in websites:
#     pages_menu = requests.get(website)
#     #pages_menu.content
#     pages = BeautifulSoup(pages_menu.content, 'html.parser')
#     pages=pages.find('ol', class_='row')
#     pages=pages.find_all('a')
#     books_links_list=set([page.get('href') for page in pages])
#     books_links=['https://books.toscrape.com/catalogue/'+ link for link in books_links_list]
    
#     for url in books_links:
#         response = requests.get(url)
#         response.content
#         soup = BeautifulSoup(response.content, 'html.parser') 
        
        
#         UPC=soup.find(class_="table table-striped").find_next(string='UPC').find_next('td').text.strip()
#         UPC_l.append(UPC)

#         title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
#         title_l.append(title)

#         price=soup.find(string='Price (incl. tax)').find_next('td').text.strip()
#         price=price.replace('£', '')
#         price=float(price)
#         price_l.append(price)

#         genre=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
#         genre_l.append(genre)

#         rating_search=soup.find('p', class_='star-rating')
#         rating_corelation={'One': 1,
#                            'Two': 2,
#                            'Three': 3,
#                            'Four': 4,
#                            'Five': 5}
#         rating_2=rating_search.get('class')[1].strip()
#         rating=rating_corelation.get(rating_2)
#         rating=int(rating)
#         rating_l.append(rating)

#         availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
#         availability=int(availability.split()[2].replace('(', ''))
#         availability_l.append(availability)

#         description=soup.find('div', class_='sub-header').find_next('p').text
#         description_l.append(description)
# df=pd.DataFrame({'upc': UPC_l,
#                  'title': title_l,
#                  'price': price_l,
#                  'rating': rating_l,
#                  'genre': genre_l,
#                  'availability': availability_l,
#                  'description': description_l })
# df

In [17]:
# UPC_l=[]
# title_l=[]
# price_l=[]
# rating_l=[]
# genre_l=[]
# availability_l=[]
# description_l=[]
# for website in websites:
#     pages_menu = requests.get(website)
#     #pages_menu.content
#     pages = BeautifulSoup(pages_menu.content, 'html.parser')
#     pages=pages.find('ol', class_='row')
#     pages=pages.find_all('a')
#     books_links_list=set([page.get('href') for page in pages])
#     books_links=['https://books.toscrape.com/catalogue/'+ link for link in books_links_list]
    
#     for url in books_links:
#         response = requests.get(url)
#         response.content
#         soup = BeautifulSoup(response.content, 'html.parser') 

#         price=soup.find(string='Price (incl. tax)').find_next('td').text.strip()
#         price=price.replace('£', '')
#         price=float(price)
#         # print(f'price {price}')
        
#         rating_search=soup.find('p', class_='star-rating')
#         rating_corelation={'One': 1,
#                            'Two': 2,
#                            'Three': 3,
#                            'Four': 4,
#                            'Five': 5}
#         rating_2=rating_search.get('class')[1].strip()
#         rating=rating_corelation.get(rating_2)
#         rating=int(rating)
#         # print(f'rating {rating}')
        
#         if price<=20 and rating>=4:
#             price_l.append(price)
#             rating_l.append(rating)
            
#             UPC=soup.find(class_="table table-striped").find_next(string='UPC').find_next('td').text.strip()
#             UPC_l.append(UPC)
    
#             title=soup.find(class_="col-sm-6 product_main").find_next('h1').text.strip()
#             title_l.append(title)
    
#             genre=soup.find('ul', class_='breadcrumb').find_next('li').find_next('li').find_next('li').text.strip()
#             genre_l.append(genre)
        
#             availability=soup.find(class_='table table-striped').find_next('th', string='Availability').find_next('td').text
#             availability=int(availability.split()[2].replace('(', ''))
#             availability_l.append(availability)
    
#             description=soup.find('div', class_='sub-header').find_next('p').text
#             description_l.append(description)
            
# df=pd.DataFrame({'upc': UPC_l,
#                  'title': title_l,
#                  'price': price_l,
#                  'rating': rating_l,
#                  'genre': genre_l,
#                  'availability': availability_l,
#                  'description': description_l })
# df

In [18]:
# print(UPC_l, title_l, price_l, rating_l, genre_l, availability_l, description_l)

In [19]:
# UPC_l
# title_l
# price_l
# rating_l
# # genre_l
# # availability_l
# # description_l

In [20]:
# df=pd.DataFrame({'upc': UPC_l,
#                  'title': title_l,
#                  'price': price_l,
#                  'rating': rating_l,
#                  'genre': genre_l,
#                  'availability': availability_l,
#                  'description': description_l })

In [21]:
# df

In [22]:
# pages_menu = requests.get('https://books.toscrape.com/catalogue/page-1.html')
# #pages_menu.content
# pages = BeautifulSoup(pages_menu.content, 'html.parser')
# pages=pages.find('ol', class_='row')
# pages=pages.find_all('a')
# books_links_list=set([page.get('href') for page in pages])
# books_links=['https://books.toscrape.com/catalogue/'+ link for link in books_links_list]
# books_links
# UPC=soup.find(class_="table table-striped").find_next(string='UPC').find_next('td').text.strip()
# UPC

In [23]:
# pages=pages.find('ol', class_='row')
# pages_2=pages.find_all('a')
# #books_links_list=[page.get('href') for page in pages]
# books_links_list=[page.get('href') for page in pages_2]
# books_links_set=set((books_links_list))
# books_links=['https://books.toscrape.com/'+ link for link in books_links_list]

In [24]:
# pages=pages.find('ol', class_='row')
# pages=pages.find_all('li')
# books_links_list=[page.get('href') for page in pages]
# books_links=['https://books.toscrape.com/'+ link for link in books_links_list]

# 1. Find the parent table
table = soup.find('table', class_='table table-striped')

# 2. Check if the table exists
if table:
    # 3. Find the header containing the specific text (using string= is safer)
    upc_header = table.find('th', string='UPC')
    
    # 4. Check if the UPC header was found
    if upc_header:
        # 5. Find the next sibling data cell and extract text
        upc_data_cell = upc_header.find_next_sibling('td')
        UPC = upc_data_cell.text.strip()
    else:
        print("UPC header not found in the table.")
        UPC = None # Set to None or empty string if not found
else:
    print("Product Information table not found on the page.")
    UPC = None